In [ ]:
import torch
import argparse
import pandas as pd
import matplotlib.pyplot as plt

from models import timer_xl

: 

In [ ]:
# init the args of pre-trained Timer-XL

args = argparse.Namespace()
args.input_token_len = 96
args.output_token_len = 96
args.d_model = 1024
args.n_heads = 8
args.e_layers = 8
args.d_ff = 2048
args.dropout = 0.1
args.activation = 'relu'
args.use_norm = True
args.flash_attention = False
args.covariate = False
args.output_attention = False

model = timer_xl.Model(args)

# download the checkpoint from https://cloud.tsinghua.edu.cn/f/01c35ca13f474176be7b/
model.load_state_dict(torch.load('checkpoint.pth')) 

In [ ]:
# evaluate zere-shot prediction

df = pd.read_csv("https://raw.githubusercontent.com/WenWeiTHU/TimeSeriesDatasets/refs/heads/main/ETT-small/ETTh2.csv")

lookback_length = 1440      # support the maximum context length up to 2880 
input = torch.tensor(df["OT"][:lookback_length]).unsqueeze(0).float()

# generate forecast 
prediction_length = 96     # forecast the next 96 timestamps, supporting maximum prediction length encompassed in the context length

output =  model(input.unsqueeze(-1), None, None)

In [ ]:
plt.plot(input.squeeze().detach().numpy())

In [ ]:
plt.plot(output.squeeze().detach().numpy())

In [ ]:
# Note that the output is the whole sequence of next token prediction
# so we need to select the last token (token_len=96) as the final prediction

pred = output[:, -96:, 0].squeeze().detach().numpy()

In [ ]:
# plot the prediction

plt.figure(figsize=(12, 4))
plt.plot(df["OT"][:lookback_length + prediction_length], color="limegreen", label="Groundtruth")
plt.plot(range(lookback_length, lookback_length + prediction_length), pred, color="tomato", label="Prediction")
plt.plot(df["OT"][:lookback_length], color="royalblue", label="Lookback")
plt.legend()
plt.grid()
plt.show()